In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd 
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df=df.copy().drop(columns=['EIN','NAME'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [21]:
sa= df['APPLICATION_TYPE'].value_counts()
sa= sa[sa < 1065]
sa=sa.copy()
def update_app(value):
    if value in sa:
        return 'OTHER'
    else:
        return value
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(update_app)
df= df.drop(df[df['APPLICATION_TYPE'] == 'OTHER'].index)
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
Name: APPLICATION_TYPE, dtype: int64

In [22]:
sc= df['CLASSIFICATION'].value_counts().copy()
sc= sc[sc < 1918]
sc=sc.copy()
def update_app(value):
    if value in sc:
        return 'OTHER'
    else:
        return value
df['CLASSIFICATION'] = df['CLASSIFICATION'].apply(update_app)
df= df.drop(df[df['CLASSIFICATION'] == 'OTHER'].index)
df['CLASSIFICATION'].value_counts()

C1000    15093
C2000     6045
C1200     4836
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [23]:
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [24]:
dt = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT']

dummies = pd.get_dummies(df[dt])
df= pd.concat([df, dummies], axis=1)
df= df.drop(columns=dt)

In [25]:
X=df.drop(columns='IS_SUCCESSFUL')
y=df['IS_SUCCESSFUL']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [26]:
scaler = StandardScaler()
xs= scaler.fit(X_train)
X_train_scaled = xs.transform(X_train)
X_test_scaled = xs.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=120, input_dim=36, activation='relu'))

nn.add(tf.keras.layers.Dense(units=100, activation='relu'))
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn.add(tf.keras.layers.Dense(units=70, activation='relu'))
nn.add(tf.keras.layers.Dense(units=100, activation='tanh'))

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [28]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               4440      
                                                                 
 dense_1 (Dense)             (None, 100)               12100     
                                                                 
 dense_2 (Dense)             (None, 80)                8080      
                                                                 
 dense_3 (Dense)             (None, 70)                5670      
                                                                 
 dense_4 (Dense)             (None, 100)               7100      
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 37491 (146.45 KB)
Trainable params: 37491 

In [29]:
nn.compile( loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [30]:
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
654/654 [==============================] - 5s 4ms/step - loss: 0.5808 - accuracy: 0.7106
Epoch 2/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5692 - accuracy: 0.7193
Epoch 3/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5680 - accuracy: 0.7210
Epoch 4/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5653 - accuracy: 0.7229
Epoch 5/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5633 - accuracy: 0.7213
Epoch 6/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5638 - accuracy: 0.7223
Epoch 7/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5619 - accuracy: 0.7220
Epoch 8/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5613 - accuracy: 0.7232
Epoch 9/100
654/654 [==============================] - 2s 4ms/step - loss: 0.5605 - accuracy: 0.7237
Epoch 10/100
654/654 [==============================] - 3s 4ms/step - loss: 0.5601 - accura

In [31]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

218/218 - 1s - loss: 0.5881 - accuracy: 0.7107 - 689ms/epoch - 3ms/step
Loss: 0.5880825519561768, Accuracy: 0.7107414603233337


In [32]:
# Export our model to HDF5 file
import h5py
hdf5_filename = 'AlphabetSoupCharity_Optimization.h5'
nn.save(hdf5_filename)
print('Model saved to : ',hdf5_filename)

Model saved to :  AlphabetSoupCharity_Optimization.h5


c:\Users\pammo\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
